### Y ahora - Semana 3 Día 3

## AutoGen Core

Algo un poco diferente.

Esto es independiente del marco de agente subyacente.

Puede usar AutoGen AgentChat o cualquier otra opción; es un marco de interacción con agentes.

Desde ese punto de vista, su posicionamiento es similar al de LangGraph.

### El principio fundamental

Autogen Core desvincula la lógica de un agente de la forma en que se entregan los mensajes.
El marco proporciona una infraestructura de comunicación, junto con el ciclo de vida del agente, y los agentes son responsables de su propio trabajo.

La infraestructura de comunicación se denomina entorno de ejecución.

Hay dos tipos: **autónomo** y **distribuido**.

Hoy usaremos un entorno de ejecución independiente: **SingleThreadedAgentRuntime**, una implementación local integrada del entorno de ejecución del agente.

Mañana analizaremos brevemente un entorno de ejecución distribuido.


In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv(override=True)


True

### Primero definimos nuestro objeto Mensaje.

La estructura que queramos para los mensajes en nuestro framework de Agente.

In [2]:
# Vamos a tener uno simple!

@dataclass
class Message:
    content: str


### Ahora definimos nuestro Agente.

Una subclase de RoutedAgent.

Cada Agente tiene un **ID de Agente** que consta de dos componentes:
- `agent.id.type` describe el tipo de agente y
- `agent.id.key` le proporciona su identificador único.

Cualquier método con la propiedad `@message_handler` podrá recibir mensajes.


In [3]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("Simple")

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"Este es un mensaje de {self.id.type}-{self.id.key}. Tu dijiste '{message.content}' y yo no estoy de acuerdo.")

### Bien, creemos un entorno de ejecución independiente y registremos nuestro tipo de agente.

In [4]:

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

AgentType(type='simple_agent')

### ¡Muy bien! Iniciemos un tiempo de ejecución y enviemos un mensaje.

In [5]:
runtime.start()

In [6]:
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Bien, ¡aquí estoy!"), agent_id)
print(">>>", response.content)

>>> Este es un mensaje de simple_agent-default. Tu dijiste 'Bien, ¡aquí estoy!' y yo no estoy de acuerdo.


In [7]:
await runtime.stop()
await runtime.close()

### Bien, ahora hagamos algo más interesante.

¡Usaremos un asistente de AgentChat!

In [8]:

class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} ha recibido un mensaje: {message.content}")
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} respondió: {reply}")
        return Message(content=reply)
    


In [9]:
from autogen_core import SingleThreadedAgentRuntime

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

AgentType(type='LLMAgent')

In [10]:
runtime.start()  #Comience a procesar mensajes en segundo plano.
response = await runtime.send_message(Message("Hola, ¡qué tal!"), AgentId("LLMAgent", "default"))
print(">>>", response.content)
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

LLMAgent ha recibido un mensaje: Hola, ¡qué tal!
LLMAgent respondió: ¡Hola! Estoy aquí para ayudarte. ¿En qué puedo asistirte hoy?
>>> ¡Hola! Estoy aquí para ayudarte. ¿En qué puedo asistirte hoy?
>>> Este es un mensaje de simple_agent-default. Tu dijiste '¡Hola! Estoy aquí para ayudarte. ¿En qué puedo asistirte hoy?' y yo no estoy de acuerdo.
LLMAgent ha recibido un mensaje: Este es un mensaje de simple_agent-default. Tu dijiste '¡Hola! Estoy aquí para ayudarte. ¿En qué puedo asistirte hoy?' y yo no estoy de acuerdo.
LLMAgent respondió: Entiendo, lamento si mi respuesta no fue la esperada. Estoy aquí para escucharte y ayudarte en lo que necesites. ¿Cómo puedo mejorar mi respuesta o asistirte de otra manera?


In [11]:
await runtime.stop()
await runtime.close()

### Bien, ahora vamos a mostrar esto en funcionamiento: ¡hagamos que 3 agentes interactúen!

In [12]:
from autogen_ext.models.ollama import OllamaChatCompletionClient


class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OllamaChatCompletionClient(model="llama3.2", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [13]:
JUDGE = "Eres un juez de un juego de piedra, papel o tijera. Los jugadores han hecho estas elecciones:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = "Eres un jugador de piedra, papel o tijera. Responde solo con una palabra, una de las siguientes: piedra, papel o tijera."
        message = Message(content=instruction)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message, inner_1)
        response2 = await self.send_message(message, inner_2)
        result = f"Jugador 1: {response1.content}\nJugador 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}¿Quién gana?"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)


In [14]:
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [15]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="Empezamos")
response = await runtime.send_message(message, agent_id)
print(response.content)

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

In [15]:
await runtime.stop()
await runtime.close()